## 导入库和数据

In [33]:
import numpy as np
import pandas as pd
import sys
import seaborn as sns
from time import sleep
import matplotlib.pyplot as plt
on_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_online_stage1_train\ccf_online_stage1_train.csv')
off_train=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_train\ccf_offline_stage1_train.csv')
# oftid_train=pd.read_csv(r'D:\Data\TCForNewComer\off_train_sameUser_id_test.csv')
test=pd.read_csv(r'D:\Data\TCForNewComer\ccf_offline_stage1_test_revised.csv')
samplt=pd.read_csv(r'D:\Data\TCForNewComer\sample_submission.csv')

In [34]:
off_train=off_train.sort_values(by=['User_id'])
off_train.index=np.arange(0,len(off_train),1)
off_train.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,4,1433,8735,30:5,10,20160214,null
1,4,1469,2902,0.95,10,20160607,null
2,35,3381,1807,300:30,0,20160130,null
3,35,3381,9776,10:5,0,20160129,null
4,35,3381,11951,200:20,0,20160129,null


In [35]:
test.sort_values(by=['User_id'])
test['Date_received']=test['Date_received'].astype(np.int64)
test['Date_received']=test['Date_received'].apply(lambda x: str(x))
test.index=np.arange(0,len(test),1)
test.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received
0,4129537,450,9983,30:5,1,20160712
1,6949378,1300,3429,30:5,null,20160706
2,2166529,7113,6928,200:20,5,20160727
3,2166529,7113,1808,100:10,5,20160727
4,6172162,7605,6500,30:1,2,20160708


In [36]:
on_train=on_train.sort_values(by=['User_id'])
on_train.index=np.arange(0,len(on_train),1)
on_train.head()

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,4,25104,2,100145044,100:10,20160331,null
1,4,45612,1,null,null,null,20160308
2,36,46701,0,null,null,null,20160120
3,64,11200,0,null,null,null,20160526
4,64,29214,0,null,null,null,20160606


In [41]:
on_train.loc[[0,1,2,3]]

,User_id,Merchant_id,Action,Coupon_id,Discount_rate,Date_received,Date
0,4,25104,2,100145044,100:10,20160331,null
1,4,45612,1,null,null,null,20160308
2,36,46701,0,null,null,null,20160120
3,64,11200,0,null,null,null,20160526


In [ ]:
col='User_id'
a=set(on_train[col].values)
b=set(test[col].values)
c=set(oftid_train[col].values)
print(len(b))
print(len(a))
print(len(c))

print(len(a&b))
print(len(b&c))
print(len(c-b))

## 增加一列之前没用优惠券是否购买过相同商品
- 可能是日常用品，或者零食
- 可能是耐用品，然后买了之后就不会再买

In [ ]:
indexs=oftid_train[(oftid_train['Coupon_id']=='null')&(oftid_train['Date']!='null')].index
odinary=oftid_train.iloc[indexs]

In [ ]:
oftid_train.drop(indexs,axis=0,inplace=True)

In [ ]:
test['ord_buy']=0#增加一列

In [ ]:
test.index=np.arange(0,len(test),1)
odinary.index=np.arange(0,len(odinary),1)
oftid_train.index=np.arange(0,len(oftid_train),1)

In [ ]:
#计算同一件商品半年内普通购买次数
from ipykernel import kernelapp as app
i=0
j=10
length=len(test)
for index in test.index:
    rate=int(i*100/length)#看进度
    if (rate in list(np.arange(10.,110.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10

    uid=test.iloc[index]['User_id']
    mid=test.iloc[index]['Merchant_id']
    if len(odinary[odinary['User_id']==uid].index)!=0:
        for ind in odinary[odinary['User_id']==uid].index:
            if odinary.iloc[ind]['Merchant_id']==mid:
                test.loc[index,'ord_buy']+=1
    i+=1

In [ ]:
test.to_csv(r'D:\Data\TCForNewComer\deal\test_addordinarybuy.csv',index=False)

## 增加用券在线购买和普通购买、领取优惠券的列

In [ ]:
click_index=on_train[on_train['Action']==0].index
on_train=on_train.drop(click_index,axis=0)

In [ ]:
a=set(on_train['User_id'].values)
b=set(test['User_id'].values)
print(len(b))
print(len(a))
print(len(a&b))

In [ ]:
#删除on_train中的User_id不一样的行
row=[]
i=0
j=10
length=len(a&b)
for i in (a&b):
    rate=int(i*100/length)
    if (rate in list(np.arange(10.,110.,10.)))&(rate==j):
        print('.'*int(rate/10),rate,'%')
        j=j+10
    row.extend(list(off_train[off_train['User_id']==i].index))
    i+=1
on_train.drop(row,axis=0,inplace=True)

on_train.to_csv(r'D:\Data\TCForNewComer\deal\on0_train_sameUser_id_test.csv',index=False)

## 选取间隔日期大于15天，标记为负样本

In [ ]:
p_train=off_train[(off_train['Coupon_id']!='null')&(off_train['Date']!='null')]#正样本
n_train=off_train[(off_train['Coupon_id']!='null')&(off_train['Date']=='null')]#负样本

#转换为时间格式
p_train['Date_received']=pd.to_datetime(p_train['Date_received'])
p_train['Date']=pd.to_datetime(p_train['Date'])
n_train['Date_received']=pd.to_datetime(n_train['Date_received'])
test['Date_received']=pd.to_datetime(test['Date_received'])

p_train=p_train[((p_train['Date']-p_train['Date_received']).astype('timedelta64[D]'))<=15]#保留拿到和使用日期间隔小于15天的数据

## 找出正样本和负样本中的Merchant_id列和测试集相同的数据

In [ ]:
p_train=p_train.sort_values(by=['User_id'])
n_train=n_train.sort_values(by=['User_id'])

In [ ]:
col='Merchant_id'
p=set(p_train[col].values)
t=set(test[col].values)
n=set(n_train[col].values)
print(len(p))
print(len(n))
print(len(t))

In [ ]:
p_t=list(p&t)
n_t=list(n&t)
print(len(p_t))
print(len(n_t))

In [ ]:
p_tdf=pd.DataFrame()
for i in p_t:
    daf=p_train[p_train[col]==i]
    p_tdf=pd.concat([p_tdf,daf],axis=0)

In [ ]:
n_tdf=pd.DataFrame()
for i in n_t:
    daf=n_train[n_train[col]==i]
    n_tdf=pd.concat([n_tdf,daf],axis=0)

## 使用普通购买的数据

In [ ]:
col='User_id'
p=set(p_train[col].values)
n=set(n_train[col].values)
print(len(p))
print(len(n))

In [ ]:
p_tdf=pd.DataFrame()
for i in (p|n):
    daf=off_train[(off_train[col]==i)&(off_train['Date']!='')]
    p_tdf=pd.concat([p_tdf,daf],axis=0)

## 删除负样本中Coupon_id列的值不在测试集中的行

In [ ]:
n_tdf['Coupon_id']=n_tdf['Coupon_id'].astype(np.int64)
test['Coupon_id']=test['Coupon_id'].astype(np.int64)

In [ ]:
col='Coupon_id'
bincol=[]*len(n_tdf)
n1=set(n_tdf[col].values)
t1=set(test[col].values)
print(len(n1))
print(len(t1))

In [ ]:
no_exist_test=n1-t1
for i in no_exist_test:
    n_tdf.drop(n_tdf[n_tdf[col]==i].index,axis=0,inplace=True)

In [ ]:
p_tdf['Date_received']=p_tdf['Date_received'].apply(lambda x: x.weekday()+1)#转换为星期几
n_tdf['Date_received']=n_tdf['Date_received'].apply(lambda x: x.weekday()+1)
test['Date_received']=test['Date_received'].apply(lambda x: x.weekday()+1)
#正样本为1，负样本为0
p_tdf.loc[:,'Date']=1
n_tdf.loc[:,'Date']=0

train=pd.concat([p_tdf,n_tdf],axis=0)
train=train.sort_values(by=['User_id'])

In [ ]:
train['Date_received']=train['Date_received'].apply(lambda x: 1 if x<=5 else 0)
test['Date_received']=test['Date_received'].apply(lambda x: 1 if x<=5 else 0)

In [ ]:
train=train.replace('null',np.nan)
test=test.replace('null',np.nan)

In [ ]:
train['Coupon_id']=train['Coupon_id'].astype(np.int64)

## 测试

In [ ]:
train['Merchant_id']=train['Merchant_id'].astype(np.object)
train['User_id']=train['User_id'].astype(np.object)
train['Date_received']=train['Date_received'].astype('category')
test['Merchant_id']=test['Merchant_id'].astype(np.object)
test['Coupon_id']=test['Coupon_id'].astype(np.object)
test['Distance']=test['Distance'].astype(np.object)
test['Date_received']=test['Date_received'].astype('category')

In [ ]:
test['User_id']=test['User_id'].astype(np.object)

# 转化满减为小数折扣

In [ ]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return round((float(x.split(':')[0])-float(x.split(':')[1]))/float(x.split(':')[0]),2)

In [ ]:
def Discount_rate(x):#转化函数
    if x.startswith('0'):
        return float(x)
    else:
        return int(x.split(':')[0])-int(x.split(':')[1])

In [ ]:
#将满减转化为小数
train['Discount_rate']=train['Discount_rate'].apply(Discount_rate)

## 填充训练集的空缺值

In [ ]:
train=train.fillna(method='ffill')

In [ ]:
train['Distance']=train['Distance'].astype(np.int64)

In [ ]:
train.isnull().any()

In [ ]:
train.info()

### 填充测试集的空缺值

In [ ]:
test=test.fillna(method='ffill')

In [ ]:
test['Discount_rate']=test['Discount_rate'].apply(Discount_rate)

In [ ]:
# test['Merchant_id']=test['Merchant_id'].astype(np.int64)
# test['Coupon_id']=test['Coupon_id'].astype(np.int64)
test['Distance']=test['Distance'].astype(np.int64)

In [ ]:
test.info()

## 删除没有购买行为的优惠券数据

In [ ]:
for j in list(set(train['Coupon_id'].values)):
        df_1=train[train['Coupon_id']==j]
        if len(np.unique(df_1['Date']))==1:
            for i in train[train['Coupon_id']==j].index:
                train['Coupon_id'].drop(i,axis=0)

In [ ]:
#类别形变量转换为指示性变量
col='Date_received'
new_dummies=pd.get_dummies(train[col],prefix=col)
train=pd.concat([train,new_dummies],axis=1)
train.drop([col],axis=1,inplace=True)

new_dummies=pd.get_dummies(test[col],prefix=col)
test=pd.concat([test,new_dummies],axis=1)
test.drop([col],axis=1,inplace=True)

len(test.columns)

## 距离统计分布

In [ ]:
import seaborn as sns 
%matplotlib inline

sns.countplot(x='Distance',hue='Date',data=train)

In [ ]:
import seaborn as sns
%matplotlib inline

sns.countplot(x='Date_received',hue='Date',data=train)

In [ ]:
import seaborn as sns 
%matplotlib inline
plt.figure(figsize=(10,6))
sns.countplot(x='Discount_rate',hue='Date',data=train)
plt.xticks(rotation=90)
plt.show()

In [ ]:
train.head()

In [ ]:
a=set(train['Coupon_id'].values)
c=set(test['Coupon_id'].values)
len(c)

In [ ]:
corr=train.corr()
corr

In [ ]:
combine=[train,test]
for data in combine:
    data.drop(['Date_received'],axis=1,inplace=True)

In [ ]:
train.head()

In [ ]:
a=set(train['Merchant_id'].values)
c=set(test['Merchant_id'].values)
len(c)

In [ ]:
len(a&c)

# 训练模型

In [ ]:
K=3
from sklearn.metrics import roc_auc_score,auc
from time import sleep
import winsound
from sklearn.model_selection import StratifiedKFold
# kf = KFold(n_splits = K,random_state = 1,shuffle = True)
kf=StratifiedKFold(n_splits = K,random_state = 90,shuffle = True)
X=train.drop(['Date'],axis=1)
y=train['Date']

## 归一化

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
x_predict=test
# 1. min max scaler
min_max_scaler = MinMaxScaler()
x_predict_min_max_scaled = min_max_scaler.fit_transform(x_predict)

# 2. standard scaler
std_scaler = StandardScaler()
x_predict_std_scaled = std_scaler.fit_transform(x_predict)

In [ ]:
# 1. min max scaler
min_max_scaler = MinMaxScaler()
# x_train_min_max_scaled = min_max_scaler.fit_transform(x_train)
# x_test_min_max_scaled = min_max_scaler.transform(x_test)
X_test_min_max_scaled=min_max_scaler.fit_transform(X)#整个数据集
# 2. standard scaler
std_scaler = StandardScaler()
# x_train_std_scaled = std_scaler.fit_transform(x_train)
# x_test_std_scaled = std_scaler.transform(x_test)
X_test_std_scaled=std_scaler.fit_transform(X)

In [ ]:
import xgboost as xgb
#全部  0.5552 learning_rate=0.09
model=xgb.XGBClassifier(n_estimators=200,
                        max_depth=2,#6
                        objective="binary:logistic",
                        learning_rate=0.09, 
                        subsample=.8,
                        min_child_weight=6,
                        colsample_bytree=.4,
                        scale_pos_weight=1.6,
                        gamma=9,
                        seed=100,
                        reg_alpha=8,
                        reg_lambda=1.3)

In [ ]:
def Avg_auc(pred):
    aucc=0
    i=0
    df=train.copy()
    df['y_valid_pred']=pred
    for j in list(set(df['Coupon_id'].values)):
        df_1=df[df['Coupon_id']==j]
        if len(np.unique(df_1['Date']))==1:
            continue
        aucc=aucc+roc_auc_score(df_1['Date'],df_1['y_valid_pred'])
        i+=1
    aucc=aucc/i
    df.drop(['y_valid_pred'],axis=1,inplace=True)
    return aucc

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [ ]:
from sklearn.ensemble import AdaBoostClassifier# 
model=AdaBoostClassifier(n_estimators=400,learning_rate=1,random_state=100)
model.fit(X,y)
pred=model.predict_proba(X)[:,1]
print('Avg_auc for full training set:',Avg_auc(pred))

In [ ]:
y_test_pred=0
y_valid_pred=0*y
fit_model=[]
# XMIN=pd.DataFrame(X_test_std_scaled,columns=['Merchant_id', 'Coupon_id', 'Discount_rate', 'Distance', 'Date_received'])
for i,(train_index,test_index) in enumerate(kf.split(X,y)):
    y_train,y_valid=y.iloc[train_index],y.iloc[test_index]
    x_train,x_valid=X.iloc[train_index],X.iloc[test_index]
    print('\nFlod:',i)
    fitted_model=model.fit(x_train,y_train)
    pred= fitted_model.predict_proba(x_valid)[:,1]
    y_valid_pred.iloc[test_index]=pred
    print(pd.DataFrame(pred).head(1))
    
    y_test_pred+=fitted_model.predict_proba(test)[:,1]
    fit_model.append(fitted_model)
    winsound.Beep(600,1000)
    sleep(1)
y_test_pred/=K
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))
# sub=pd.DataFrame({'id':test_id,'target':y_test_pred})
# sub.to_csv(r'D:\Data\Porto Seguro s Safe Driver Prediction\result\123456KFlod_submission.csv',float_format='%.4f',index=False)

In [ ]:
y_train['a']=np.random.normal(len(y_train))

In [ ]:
print('Avg_auc for full training set:',Avg_auc(y_valid_pred))

In [ ]:
import numpy as np
from sklearn.metrics import roc_auc_score
y_true = np.array([0, 1, 0, 1])
y_scores = np.array([0.1, 0.4, 0.35, 0.8])
y_true.astype(np.int64)

In [ ]:
train.head()